In [1]:
import org.apache.spark.sql.{SQLContext, SaveMode}
import org.apache.spark.sql.types.{StructType, StructField, StringType, DoubleType};
import com.databricks.spark.xml._

val sqlContext = new SQLContext(sc)
import sqlContext.implicits._
sqlContext.sql("set spark.sql.caseSensitive=true") 
spark.sparkContext.uiWebUrl

Waiting for a Spark session to start...

sqlContext = org.apache.spark.sql.SQLContext@78115553


Some(http://iuni2:4041)

In [2]:
val WoS = spark.read.format("parquet").load("/WoSraw/parquet/*.parquet")

val WoS2 = WoS.select("UID","static_data.summary.titles","static_data.fullrecord_metadata.category_info","static_data.summary.names.name")
WoS2.printSchema

root
 |-- UID: string (nullable = true)
 |-- titles: struct (nullable = true)
 |    |-- _count: long (nullable = true)
 |    |-- title: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- _VALUE: string (nullable = true)
 |    |    |    |-- _translated: string (nullable = true)
 |    |    |    |-- _type: string (nullable = true)
 |-- category_info: struct (nullable = true)
 |    |-- headings: struct (nullable = true)
 |    |    |-- _count: long (nullable = true)
 |    |    |-- heading: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |-- subheadings: struct (nullable = true)
 |    |    |-- _count: long (nullable = true)
 |    |    |-- subheading: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |-- subjects: struct (nullable = true)
 |    |    |-- _count: long (nullable = true)
 |    |    |-- subject: array (nullable = true)
 |    |    |    |-- element: struct (con

WoS = [UID: string, _r_id_disclaimer: string ... 13 more fields]
WoS2 = [UID: string, titles: struct<_count: bigint, title: array<struct<_VALUE:string,_translated:string,_type:string>>> ... 2 more fields]


[UID: string, titles: struct<_count: bigint, title: array<struct<_VALUE:string,_translated:string,_type:string>>> ... 2 more fields]

In [2]:
import scala.util.Try
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{Row, SQLContext}
import org.apache.spark.sql.functions.countDistinct
import org.apache.spark.sql.SaveMode

val session = SparkSession.builder().getOrCreate()

val azureFile = sc.textFile("/azureBlob.txt")
session.conf.set(
    "fs.azure.account.key.iunimag.blob.core.windows.net",
     azureFile.collect().mkString
)

//val magPapers = spark.read.format("csv")
      //.option("header", "true")
//      .option("delimiter","\t")
//      .option("inferSchema","true")
//      .load("wasbs://mag-2020-09-01@iunimag.blob.core.windows.net/mag/Papers.txt")
//magPapers.show

session = org.apache.spark.sql.SparkSession@4c9bb366
azureFile = /azureBlob.txt MapPartitionsRDD[1] at textFile at <console>:45


/azureBlob.txt MapPartitionsRDD[1] at textFile at <console>:45

In [39]:
val magPapers2 = magPapers.select("_c0","_c2","_c3","_c4","_c6","_c7","_c8","_c11","_c12","_c13","_c19","_c20","_c22")
.toDF("MAGid","Doi","DocType","PaperTitle","BookTitle","Year","Date","JournalId","ConferenceSeriesId","ConferenceInstanceId","CitationCount","EstimatedCitation","FamilyId")
val magPapers3 = magPapers2.filter($"Year" >= "1989" && $"Year" <= "2017").filter($"DocType" === "Journal" || $"DocType" === "Conference" || $"DocType" === "BookChapter").cache()
magPapers3.describe().filter($"summary" === "count").show

+-------+--------+--------+--------+----------+---------+--------+---------+------------------+--------------------+-------------+-----------------+--------+
|summary|   MAGid|     Doi| DocType|PaperTitle|BookTitle|    Year|JournalId|ConferenceSeriesId|ConferenceInstanceId|CitationCount|EstimatedCitation|FamilyId|
+-------+--------+--------+--------+----------+---------+--------+---------+------------------+--------------------+-------------+-----------------+--------+
|  count|64625812|48176760|64625812|  64625812|  3890193|64625812| 58694317|           4033251|             1401647|     64625809|         64625809| 1207899|
+-------+--------+--------+--------+----------+---------+--------+---------+------------------+--------------------+-------------+-----------------+--------+



magPapers2 = [MAGid: bigint, Doi: string ... 11 more fields]
magPapers3 = [MAGid: bigint, Doi: string ... 11 more fields]


[MAGid: bigint, Doi: string ... 11 more fields]

In [43]:
val magPapersF = magPapers3.filter("FamilyId is not null")
magPapersF.describe().filter($"summary" === "count").show

+-------+-------+------+-------+----------+---------+-------+---------+------------------+--------------------+-------------+-----------------+--------+
|summary|  MAGid|   Doi|DocType|PaperTitle|BookTitle|   Year|JournalId|ConferenceSeriesId|ConferenceInstanceId|CitationCount|EstimatedCitation|FamilyId|
+-------+-------+------+-------+----------+---------+-------+---------+------------------+--------------------+-------------+-----------------+--------+
|  count|1207899|986412|1207899|   1207899|   115354|1207899|  1037257|            170634|               67501|      1207899|          1207899| 1207899|
+-------+-------+------+-------+----------+---------+-------+---------+------------------+--------------------+-------------+-----------------+--------+



magPapersF = [MAGid: bigint, Doi: string ... 11 more fields]


[MAGid: bigint, Doi: string ... 11 more fields]

In [46]:
val magPapersF = magPapers3.filter($"FamilyId" =!= $"MAGid")
magPapersF.describe().filter($"summary" === "count").show

+-------+------+------+-------+----------+---------+------+---------+------------------+--------------------+-------------+-----------------+--------+
|summary| MAGid|   Doi|DocType|PaperTitle|BookTitle|  Year|JournalId|ConferenceSeriesId|ConferenceInstanceId|CitationCount|EstimatedCitation|FamilyId|
+-------+------+------+-------+----------+---------+------+---------+------------------+--------------------+-------------+-----------------+--------+
|  count|254440|178078| 254440|    254440|    49422|254440|   180360|             74072|               23512|       254440|           254440|  254440|
+-------+------+------+-------+----------+---------+------+---------+------------------+--------------------+-------------+-----------------+--------+



magPapersF = [MAGid: bigint, Doi: string ... 11 more fields]


[MAGid: bigint, Doi: string ... 11 more fields]

In [47]:
val magPapers4 = magPapers3.join(magPapersF, magPapers3("MAGid")===magPapersF("MAGid"), "leftanti")
magPapers4.describe().filter($"summary" === "count").show

+-------+--------+--------+--------+----------+---------+--------+---------+------------------+--------------------+-------------+-----------------+--------+
|summary|   MAGid|     Doi| DocType|PaperTitle|BookTitle|    Year|JournalId|ConferenceSeriesId|ConferenceInstanceId|CitationCount|EstimatedCitation|FamilyId|
+-------+--------+--------+--------+----------+---------+--------+---------+------------------+--------------------+-------------+-----------------+--------+
|  count|64371372|47998682|64371372|  64371372|  3840771|64371372| 58513957|           3959179|             1378135|     64371369|         64371369|  953459|
+-------+--------+--------+--------+----------+---------+--------+---------+------------------+--------------------+-------------+-----------------+--------+



magPapers4 = [MAGid: bigint, Doi: string ... 11 more fields]


[MAGid: bigint, Doi: string ... 11 more fields]

In [3]:
val magPPA = spark.read.format("csv")
      //.option("header", "true")
      .option("delimiter","\t")
      .option("inferSchema","true")
      .load("wasbs://mag-2020-09-01@iunimag.blob.core.windows.net/mag/PaperAuthorAffiliations.txt")

magPPA.show

+---+----------+----------+---+--------------------+--------------------+
|_c0|       _c1|       _c2|_c3|                 _c4|                 _c5|
+---+----------+----------+---+--------------------+--------------------+
|  9|2632942543|      null|  1|Victoriano Perruc...|                null|
| 15| 199142497|      null|  1|     Robert Münscher|Heidelberg, Deuts...|
| 15| 680395887|      null|  2|       Julia Hormuth|Reutlingen, Deuts...|
| 23|1243978490|  79576946|  1|     Eric T Stoopler|Department of Ora...|
| 23|2582258949|      null|  3| Arthur S Kuperstein|                null|
| 23|2582736345|      null|  2|        Ying Wai Sia|                null|
| 58|2641488431|      null|  1|             I. Uray|                null|
| 58|2693978806|      null|  2|           P. Kovacs|                null|
| 79|2662843304|      null|  1|           K. Suresh|                null|
| 79|2683440697|      null|  2|     T. S. J. Norman|                null|
| 87|2676159639|      null|  1| Gabrie

magPPA = [_c0: bigint, _c1: bigint ... 4 more fields]


[_c0: bigint, _c1: bigint ... 4 more fields]

In [48]:
val magPPA2 = magPPA.toDF("MAGid2","AuthorId","AffiliationId","AuthorSequenceNumber","OriginalAuthor","OriginalAffiliation")
val magPPA3 = magPapers4.join(magPPA2, magPapers4("MAGid")===magPPA2("MAGid2"), "left").cache()
magPPA3.describe().filter($"summary" === "count").show

+-------+---------+---------+---------+----------+---------+---------+---------+------------------+--------------------+-------------+-----------------+--------+---------+---------+-------------+--------------------+--------------+-------------------+
|summary|    MAGid|      Doi|  DocType|PaperTitle|BookTitle|     Year|JournalId|ConferenceSeriesId|ConferenceInstanceId|CitationCount|EstimatedCitation|FamilyId|   MAGid2| AuthorId|AffiliationId|AuthorSequenceNumber|OriginalAuthor|OriginalAffiliation|
+-------+---------+---------+---------+----------+---------+---------+---------+------------------+--------------------+-------------+-----------------+--------+---------+---------+-------------+--------------------+--------------+-------------------+
|  count|228988583|191624300|228988583| 228988583|  9278599|228988583|212656230|          13233459|             4753483|    228988567|        228988567| 3231339|228988583|228988583|    118873273|           228988583|     228988583|          146

magPPA2 = [MAGid2: bigint, AuthorId: bigint ... 4 more fields]
magPPA3 = [MAGid: bigint, Doi: string ... 17 more fields]


[MAGid: bigint, Doi: string ... 17 more fields]

In [19]:
//magPPA3.write.mode(SaveMode.Overwrite).parquet("/dataRCSC/Fellow5-flight-collaboration/magPPA3/")
val magPPA3 = spark.read.format("parquet").load("/dataRCSC/Fellow5-flight-collaboration/magPPA3/*.parquet")
val magPPAcited = magPPA3.filter($"Year" === "1989").filter("AffiliationId is not null").select("MAGid","Year","AffiliationId")
magPPAcited.describe().filter($"summary" === "count").show

+-------+------+------+-------------+
|summary| MAGid|  Year|AffiliationId|
+-------+------+------+-------------+
|  count|968294|968294|       968294|
+-------+------+------+-------------+



magPPA3 = [MAGid: bigint, Doi: string ... 17 more fields]
magPPAcited = [MAGid: bigint, Year: string ... 1 more field]


[MAGid: bigint, Year: string ... 1 more field]

In [20]:
val magPPAciting = magPPA3.filter($"Year" >= "1989" && $"Year" <= "1992").filter("AffiliationId is not null")
                    .select("MAGid","Year","AffiliationId").toDF("MAGid2","Year2","AffiliationId2")
magPPAciting.describe().filter($"summary" === "count").show

+-------+-------+-------+--------------+
|summary| MAGid2|  Year2|AffiliationId2|
+-------+-------+-------+--------------+
|  count|4360993|4360993|       4360993|
+-------+-------+-------+--------------+



magPPAciting = [MAGid2: bigint, Year2: string ... 1 more field]


[MAGid2: bigint, Year2: string ... 1 more field]

In [6]:
val magCitations = spark.read.format("csv")
      //.option("header", "true")
      .option("delimiter","\t")
      .option("inferSchema","true")
      .load("wasbs://mag-2020-09-01@iunimag.blob.core.windows.net/mag/PaperReferences.txt")
magCitations.describe().filter($"summary" === "count").show

+-------+----------+----------+
|summary|       _c0|       _c1|
+-------+----------+----------+
|  count|1665170425|1665170425|
+-------+----------+----------+



magCitations = [_c0: bigint, _c1: bigint]


[_c0: bigint, _c1: bigint]

In [21]:
val magPPAcited2 = magPPAcited.join(magCitations, magPPAcited("MAGid")===magCitations("_c1"), "left").cache()
magPPAcited2.describe().filter($"summary" === "count").show

+-------+--------+--------+-------------+--------+--------+
|summary|   MAGid|    Year|AffiliationId|     _c0|     _c1|
+-------+--------+--------+-------------+--------+--------+
|  count|32641618|32641618|     32641618|32543079|32543079|
+-------+--------+--------+-------------+--------+--------+



magPPAcited2 = [MAGid: bigint, Year: string ... 3 more fields]


[MAGid: bigint, Year: string ... 3 more fields]

In [22]:
val magPPAcited3 = magPPAcited2.join(magPPAciting, magPPAcited2("_c0")===magPPAciting("MAGid2"), "inner").cache()
magPPAcited3.describe().filter($"summary" === "count").show

+-------+--------+--------+-------------+--------+--------+--------+--------+--------------+
|summary|   MAGid|    Year|AffiliationId|     _c0|     _c1|  MAGid2|   Year2|AffiliationId2|
+-------+--------+--------+-------------+--------+--------+--------+--------+--------------+
|  count|19301654|19301654|     19301654|19301654|19301654|19301654|19301654|      19301654|
+-------+--------+--------+-------------+--------+--------+--------+--------+--------------+



magPPAcited3 = [MAGid: bigint, Year: string ... 6 more fields]


[MAGid: bigint, Year: string ... 6 more fields]

In [32]:
val insititutionCited = magPPAcited3.groupBy("AffiliationId", "AffiliationId2").agg(countDistinct("MAGid2") as "citationCount")
                            .orderBy($"AffiliationId".asc,$"AffiliationId2".asc).toDF("citedAffiliationId","citingAffiliationId","citationCount")
insititutionCited.describe().filter($"summary" === "count").show

+-------+------------------+-------------------+-------------+
|summary|citedAffiliationId|citingAffiliationId|citationCount|
+-------+------------------+-------------------+-------------+
|  count|           8939682|            8939682|      8939682|
+-------+------------------+-------------------+-------------+



insititutionCited = [citedAffiliationId: bigint, citingAffiliationId: bigint ... 1 more field]


[citedAffiliationId: bigint, citingAffiliationId: bigint ... 1 more field]

In [33]:
insititutionCited.coalesce(1).write.mode(SaveMode.Overwrite)
    .format("com.databricks.spark.csv")
    .option("header", "true")
    .option("quote", "\"")
    .save("wasbs://mag-2020-09-01@iunimag.blob.core.windows.net/MAGinsititutionCitation2014.csv")

In [23]:
val magCitationsSelf = magPPAcited3.groupBy("MAGid", "MAGid2").agg(collect_set("AffiliationId") as "AffiliationIds",
                                                                   collect_set("AffiliationId2") as "AffiliationId2s")
                        .filter($"AffiliationIds" === $"AffiliationId2s").toDF("MAGcited","MAGciting","Affiliation0Ids","Affiliation0Id2s")
magCitationsSelf.show(false)                     

+---------+----------+---------------+----------------+
|MAGcited |MAGciting |Affiliation0Ids|Affiliation0Id2s|
+---------+----------+---------------+----------------+
|1207169  |2031138133|[170201317]    |[170201317]     |
|15183712 |2159860478|[185261750]    |[185261750]     |
|16138781 |2005882843|[137975476]    |[137975476]     |
|19505608 |215882314 |[65163767]     |[65163767]      |
|26350976 |2112056905|[187531555]    |[187531555]     |
|41085029 |2129778243|[40120149]     |[40120149]      |
|43678782 |2086161368|[34077901]     |[34077901]      |
|44685148 |2416484761|[1316902750]   |[1316902750]    |
|58804230 |2330818421|[98285908]     |[98285908]      |
|79419581 |2029237864|[1301626629]   |[1301626629]    |
|84352776 |2184031454|[1325670255]   |[1325670255]    |
|90125041 |1993162602|[24768866]     |[24768866]      |
|94779788 |2154230025|[2802742124]   |[2802742124]    |
|97961608 |2097922631|[47251452]     |[47251452]      |
|130762719|2093258424|[67415387]     |[67415387]

magCitationsSelf = [MAGcited: bigint, MAGciting: bigint ... 2 more fields]


[MAGcited: bigint, MAGciting: bigint ... 2 more fields]

In [24]:
val magPPAcited2b = magPPAcited.join(magCitationsSelf, magPPAcited("MAGid")===magCitationsSelf("MAGcited"), "left")
val magPPAcited3b = magPPAcited2b.join(magPPAciting, magPPAcited2b("MAGciting")===magPPAciting("MAGid2"), "inner").cache()
magPPAcited3b.describe().filter($"summary" === "count").show

+-------+-------+-------+-------------+--------+---------+-------+-------+--------------+
|summary|  MAGid|   Year|AffiliationId|MAGcited|MAGciting| MAGid2|  Year2|AffiliationId2|
+-------+-------+-------+-------------+--------+---------+-------+-------+--------------+
|  count|1775527|1775527|      1775527| 1775527|  1775527|1775527|1775527|       1775527|
+-------+-------+-------+-------------+--------+---------+-------+-------+--------------+



magPPAcited2b = [MAGid: bigint, Year: string ... 5 more fields]
magPPAcited3b = [MAGid: bigint, Year: string ... 8 more fields]


[MAGid: bigint, Year: string ... 8 more fields]

In [25]:
val insititutionCitedSelf = magPPAcited3b.groupBy("AffiliationId", "AffiliationId2").agg(countDistinct("MAGid2") as "citationCount")
                            .orderBy($"AffiliationId".asc,$"AffiliationId2".asc).toDF("citedAffiliationId","citingAffiliationId","citationCount")
insititutionCitedSelf.describe().filter($"summary" === "count").show

+-------+------------------+-------------------+-------------+
|summary|citedAffiliationId|citingAffiliationId|citationCount|
+-------+------------------+-------------------+-------------+
|  count|             14460|              14460|        14460|
+-------+------------------+-------------------+-------------+



insititutionCitedSelf = [citedAffiliationId: bigint, citingAffiliationId: bigint ... 1 more field]


[citedAffiliationId: bigint, citingAffiliationId: bigint ... 1 more field]

In [26]:
insititutionCitedSelf.coalesce(1).write.mode(SaveMode.Overwrite)
    .format("com.databricks.spark.csv")
    .option("header", "true")
    .option("quote", "\"")
    .save("wasbs://mag-2020-09-01@iunimag.blob.core.windows.net/MAGinsititutionCitation1989self.csv")

In [5]:
val magAffiliation = spark.read.format("csv")
      //.option("header", "true")
      .option("delimiter","\t")
      .option("inferSchema","true")
      .load("wasbs://mag-2020-09-01@iunimag.blob.core.windows.net/mag/Affiliations.txt")

val magAffiliation2 = magAffiliation.select("_c0","_c2","_c3","_c4","_c6","_c7","_c8","_c9","_c10","_c11","_c12").toDF("AffiliationId2","normalizedName","displayName","GRIDid","wiki","paperTotal","paperFamilyTotal","citationTotal", "Iso3166Code", "Latitude", "Longitude")
magAffiliation2.describe().filter($"summary" === "count").show

+-------+--------------+--------------+-----------+------+-----+----------+----------------+-------------+-----------+--------+---------+
|summary|AffiliationId2|normalizedName|displayName|GRIDid| wiki|paperTotal|paperFamilyTotal|citationTotal|Iso3166Code|Latitude|Longitude|
+-------+--------------+--------------+-----------+------+-----+----------+----------------+-------------+-----------+--------+---------+
|  count|         25785|         25785|      25785| 19026|25496|     25785|           25785|        25785|      24689|   24790|    24790|
+-------+--------------+--------------+-----------+------+-----+----------+----------------+-------------+-----------+--------+---------+



magAffiliation = [_c0: bigint, _c1: int ... 12 more fields]
magAffiliation2 = [AffiliationId2: bigint, normalizedName: string ... 9 more fields]


[AffiliationId2: bigint, normalizedName: string ... 9 more fields]

In [7]:
val magPPA4 = magPPA3.join(magAffiliation2, magPPA3("AffiliationId")===magAffiliation2("AffiliationId2"), "left").cache()
magPPA4.describe().filter($"summary" === "count").show

+-------+---------+---------+---------+----------+---------+---------+---------+------------------+--------------------+-------------+-----------------+--------+---------+---------+-------------+--------------------+--------------+-------------------+--------------+--------------+-----------+---------+---------+----------+----------------+-------------+-----------+---------+---------+
|summary|    MAGid|      Doi|  DocType|PaperTitle|BookTitle|     Year|JournalId|ConferenceSeriesId|ConferenceInstanceId|CitationCount|EstimatedCitation|FamilyId|   MAGid2| AuthorId|AffiliationId|AuthorSequenceNumber|OriginalAuthor|OriginalAffiliation|AffiliationId2|normalizedName|displayName|   GRIDid|     wiki|paperTotal|paperFamilyTotal|citationTotal|Iso3166Code| Latitude|Longitude|
+-------+---------+---------+---------+----------+---------+---------+---------+------------------+--------------------+-------------+-----------------+--------+---------+---------+-------------+--------------------+--------

magPPA4 = [MAGid: bigint, Doi: string ... 28 more fields]


[MAGid: bigint, Doi: string ... 28 more fields]

In [12]:
val insititutionCollab = magPPA4.groupBy("MAGid").agg(collect_set("AffiliationId") as "AffiliationIds", first("Year") as "Year")                                 
insititutionCollab.filter($"Year" === 2014).show

+--------+--------------------+----+
|   MAGid|      AffiliationIds|Year|
+--------+--------------------+----+
|  326638|[864159182, 30200...|2014|
| 1405896|                  []|2014|
| 2012775|         [173304897]|2014|
| 2017678|         [124055696]|2014|
| 2445120|                  []|2014|
| 2766906|          [65285256]|2014|
| 4271160|         [181877577]|2014|
| 6106527|                  []|2014|
| 6554907|         [150240954]|2014|
| 7062075|        [2801766832]|2014|
| 7521368|                  []|2014|
| 7932329|                  []|2014|
| 8322395|                  []|2014|
| 8371866|                  []|2014|
| 8938693|                  []|2014|
|12325341|                  []|2014|
|14419641|[145311948, 66946...|2014|
|16334433|           [6498739]|2014|
|16492016|                  []|2014|
|16833982|        [1280604919]|2014|
+--------+--------------------+----+
only showing top 20 rows



insititutionCollab = [MAGid: bigint, AffiliationIds: array<bigint> ... 1 more field]


[MAGid: bigint, AffiliationIds: array<bigint> ... 1 more field]

In [41]:
import scala.collection.mutable
import org.apache.spark.sql.functions._

val pairsUdf = udf((arr: mutable.Seq[BigInt]) => arr.combinations(2).toArray)
val result = insititutionCollab.withColumn("Affiliation", explode(pairsUdf($"AffiliationIds")))
val result2 = result.select($"MAGid", $"Year", $"Affiliation".getItem(0),$"Affiliation".getItem(1)).cache()
result2.show(false)

+-------+----+--------------+--------------+
|MAGid  |Year|Affiliation[0]|Affiliation[1]|
+-------+----+--------------+--------------+
|9458   |1991|130238516     |2802288015    |
|326638 |2014|864159182     |3020059144    |
|838853 |2000|138689650     |79238269      |
|838853 |2000|138689650     |12912129      |
|838853 |2000|79238269      |12912129      |
|1723107|1993|905301669     |165143802     |
|2085152|2011|159650629     |3018768319    |
|2093088|2012|153718931     |91712215      |
|2093088|2012|153718931     |190082696     |
|2093088|2012|91712215      |190082696     |
|2159076|1995|1304196042    |1925986       |
|2207358|2005|121797337     |23923803      |
|2626148|2013|27577105      |60488453      |
|3050538|2000|102335020     |592451        |
|3050538|2000|102335020     |67826202      |
|3050538|2000|592451        |67826202      |
|3644224|1995|84524832      |98635879      |
|3931102|1996|133836048     |1336856363    |
|4009996|2008|98381234      |7597260       |
|4009996|2

pairsUdf = UserDefinedFunction(<function1>,ArrayType(ArrayType(DecimalType(38,0),true),true),Some(List(ArrayType(DecimalType(38,0),true))))
result = [MAGid: bigint, AffiliationIds: array<bigint> ... 2 more fields]
result2 = [MAGid: bigint, Year: string ... 2 more fields]


[MAGid: bigint, Year: string ... 2 more fields]

In [44]:
val insititutionCollab2 = result2.groupBy("Affiliation[0]", "Affiliation[1]", "Year").agg(countDistinct("MAGid") as "paperCount")
                            .orderBy($"Year".asc,$"Affiliation[0]".asc,$"Affiliation[1]".asc)
insititutionCollab2.show

+--------------+--------------+----+----------+
|Affiliation[0]|Affiliation[1]|Year|paperCount|
+--------------+--------------+----+----------+
|         19722|     124357947|1989|         1|
|         41870|     161127581|1989|         1|
|         41870|    1294671590|1989|         1|
|         52325|      45509622|1989|         1|
|         52325|     157536573|1989|         1|
|         52325|     221977067|1989|         1|
|         80043|       6902469|1989|         1|
|         80043|      16778253|1989|         1|
|         80043|      27837315|1989|         1|
|         80043|      29607241|1989|         1|
|         80043|      30685716|1989|         1|
|         80043|      42237331|1989|         1|
|         80043|     101202996|1989|         1|
|         80043|     102197404|1989|         1|
|         80043|     124227911|1989|         1|
|         80043|     124601658|1989|         1|
|         80043|     130442723|1989|         1|
|         80043|     134493880|1989|    

insititutionCollab2 = [Affiliation[0]: decimal(38,0), Affiliation[1]: decimal(38,0) ... 2 more fields]


[Affiliation[0]: decimal(38,0), Affiliation[1]: decimal(38,0) ... 2 more fields]

In [46]:
//insititutionCollab2.describe().filter($"summary" === "count").show
insititutionCollab2.coalesce(1).write.mode(SaveMode.Overwrite)
    .format("com.databricks.spark.csv")
    .option("header", "true")
    .option("quote", "\"")
    .save("wasbs://mag-2020-09-01@iunimag.blob.core.windows.net/MAGinsititutionPair.csv")

+-------+--------------+--------------+--------+----------+
|summary|Affiliation[0]|Affiliation[1]|    Year|paperCount|
+-------+--------------+--------------+--------+----------+
|  count|      13071208|      13071208|13071208|  13071208|
+-------+--------------+--------------+--------+----------+



In [7]:
val paperIntCollab = magPPA4.groupBy("MAGid").agg(concat_ws(";", collect_list("AuthorId")) as "AuthorIds",
                                                  concat_ws(";", collect_list("AffiliationId")) as "AffiliationIds",
                                                    concat_ws(";", collect_list("Iso3166Code")) as "Iso3166Codes",
                                                    countDistinct("Iso3166Code") as "countryCount")
paperIntCollab.show

+------+--------------------+--------------------+------------+------------+
| MAGid|           AuthorIds|      AffiliationIds|Iso3166Codes|countryCount|
+------+--------------------+--------------------+------------+------------+
|  9458|250125303;5233917...|130238516;2802288...| US;US;US;US|           1|
|  9715|2346949400;198037...|            36258959|          US|           1|
| 29841|          2123350797|            99542240|          PL|           1|
| 54536|223407678;3245056...|                    |            |           0|
| 54660|          2119106503|                    |            |           0|
| 74185|          2164071203|                    |            |           0|
| 75167|2938532104;211541...|           204291657|          JP|           1|
|117760|2117199664;261418...|                    |            |           0|
|161554|2678320890;268223...|                    |            |           0|
|247114|          2998319074|                    |            |           0|

paperIntCollab = [MAGid: bigint, AuthorIds: string ... 3 more fields]


[MAGid: bigint, AuthorIds: string ... 3 more fields]

In [8]:
val magPPA5 = magPPA4.join(paperIntCollab.select("MAGid","countryCount"), Seq("MAGid"), "left")
val magPPA6 = magPPA5.withColumn("internationalFlag",$"countryCount">1)
magPPA6.show

+-----+--------------------+----------+--------------------+---------+----+-------------------+---------+------------------+--------------------+-------------+-----------------+--------+------+----------+-------------+--------------------+-----------------+--------------------+--------------+--------------------+--------------------+-------------+--------------------+----------+----------------+-------------+-----------+--------+----------+------------+-----------------+
|MAGid|                 Doi|   DocType|          PaperTitle|BookTitle|Year|               Date|JournalId|ConferenceSeriesId|ConferenceInstanceId|CitationCount|EstimatedCitation|FamilyId|MAGid2|  AuthorId|AffiliationId|AuthorSequenceNumber|   OriginalAuthor| OriginalAffiliation|AffiliationId2|      normalizedName|         displayName|       GRIDid|                wiki|paperTotal|paperFamilyTotal|citationTotal|Iso3166Code|Latitude| Longitude|countryCount|internationalFlag|
+-----+--------------------+----------+---------

magPPA5 = [MAGid: bigint, Doi: string ... 29 more fields]
magPPA6 = [MAGid: bigint, Doi: string ... 30 more fields]


[MAGid: bigint, Doi: string ... 30 more fields]

In [75]:
magPPA6.describe().filter($"summary" === "count").show

+-------+---------+---------+---------+----------+---------+---------+---------+------------------+--------------------+-------------+-----------------+--------+---------+---------+-------------+--------------------+--------------+-------------------+--------------+--------------+-----------+---------+---------+----------+----------------+-------------+-----------+---------+---------+------------+
|summary|    MAGid|      Doi|  DocType|PaperTitle|BookTitle|     Year|JournalId|ConferenceSeriesId|ConferenceInstanceId|CitationCount|EstimatedCitation|FamilyId|   MAGid2| AuthorId|AffiliationId|AuthorSequenceNumber|OriginalAuthor|OriginalAffiliation|AffiliationId2|normalizedName|displayName|   GRIDid|     wiki|paperTotal|paperFamilyTotal|citationTotal|Iso3166Code| Latitude|Longitude|countryCount|
+-------+---------+---------+---------+----------+---------+---------+---------+------------------+--------------------+-------------+-----------------+--------+---------+---------+-------------+---

In [74]:
val insititutionMAGyear = magPPA6.groupBy("AffiliationId", "Year").agg(//concat_ws(";", collect_list("MAGid2")) as "MAGids",
                                                    first("displayName") as "displayName",
                                                    first("normalizedName") as "normalizedName",
                                                    first("wiki") as "wiki",
                                                    //first("paperTotal") as "paperTotalMAG", 
                                                    //first("citationTotal") as "citationTotalMAG", 
                                                    countDistinct("MAGid") as "paperTotal", 
                                                    sum("CitationCount") as "citationTotal",
                                                    sum("EstimatedCitation") as "estimatedCitationTotal",
                                                    first("Iso3166Code") as "Iso3166Code",
                                                    first("Latitude") as "Latitude",
                                                    first("Longitude") as "Longitude",
                                                    first("GRIDid") as "GRIDid",
                                                    countDistinct(when($"internationalFlag", $"MAGid")) as "paperTotalInt",
                                                    sum(when($"internationalFlag", $"CitationCount")) as "citationTotalInt",
                                                    sum(when($"internationalFlag", $"EstimatedCitation")) as "estimatedCitationTotalInt")
insititutionMAGyear.describe().filter($"summary" === "count").show

+-------+-------------+------+-----------+--------------+------+----------+-------------+----------------------+-----------+--------+---------+------+-------------+----------------+-------------------------+
|summary|AffiliationId|  Year|displayName|normalizedName|  wiki|paperTotal|citationTotal|estimatedCitationTotal|Iso3166Code|Latitude|Longitude|GRIDid|paperTotalInt|citationTotalInt|estimatedCitationTotalInt|
+-------+-------------+------+-----------+--------------+------+----------+-------------+----------------------+-----------+--------+---------+------+-------------+----------------+-------------------------+
|  count|       402926|402955|     402926|        402926|398967|    402955|       402955|                402955|     388898|  390490|   390490|309041|       402955|          262805|                   262805|
+-------+-------------+------+-----------+--------------+------+----------+-------------+----------------------+-----------+--------+---------+------+-------------+----

insititutionMAGyear = [AffiliationId: bigint, Year: string ... 13 more fields]


[AffiliationId: bigint, Year: string ... 13 more fields]

In [76]:
insititutionMAGyear.show

+-------------+----+--------------------+--------------------+--------------------+----------+-------------+----------------------+-----------+----------+----------+-------------+-------------+----------------+-------------------------+
|AffiliationId|Year|         displayName|      normalizedName|                wiki|paperTotal|citationTotal|estimatedCitationTotal|Iso3166Code|  Latitude| Longitude|       GRIDid|paperTotalInt|citationTotalInt|estimatedCitationTotalInt|
+-------------+----+--------------------+--------------------+--------------------+----------+-------------+----------------------+-----------+----------+----------+-------------+-------------+----------------+-------------------------+
|       280994|1996|King George's Med...|king george s med...|http://en.wikiped...|         2|         47.0|                    71|         IN|26.8677158| 80.916626|grid.411275.4|            0|            null|                     null|
|       376102|1991|Agricultural Rese...|agricultura

In [78]:
insititutionMAGyear//.select("wosid","headings","subheadings","WC","SC","disciplines","subdisciplines","fractions")
    .coalesce(1).write.mode(SaveMode.Overwrite)
    .format("com.databricks.spark.csv")
    .option("header", "true")
    .option("quote", "\"")
    .save("wasbs://mag-2020-09-01@iunimag.blob.core.windows.net/MAGinsititutionMAGyear.csv")
    //.save("/dataRCSC/Fellow5-flight-collaboration/MAGinsititutionMAGyear.csv")

In [54]:
val insititutionMAG = insititutionMAGyear.groupBy("AffiliationId").agg(//concat_ws(";", collect_list("MAGid2")) as "MAGids",
                                                    first("displayName") as "displayName",
                                                    first("normalizedName") as "normalizedName",
                                                    //first("paperTotalMAG") as "paperTotalMAG", 
                                                    //first("citationTotalMAG") as "citationTotalMAG", 
                                                    sum("paperTotal") as "paperTotal", 
                                                    sum("citationTotal") as "citationTotal",
                                                    sum("estimatedCitationTotal") as "estimatedCitationTotal")
insititutionMAG.describe().filter($"summary" === "count").show

+-------+-------------+-----------+--------------+----------+-------------+----------------------+
|summary|AffiliationId|displayName|normalizedName|paperTotal|citationTotal|estimatedCitationTotal|
+-------+-------------+-----------+--------------+----------+-------------+----------------------+
|  count|        24992|      24992|         24992|     24993|        24993|                 24993|
+-------+-------------+-----------+--------------+----------+-------------+----------------------+



insititutionMAG = [AffiliationId: bigint, displayName: string ... 4 more fields]


lastException: Throwable = null


[AffiliationId: bigint, displayName: string ... 4 more fields]